# Tutoriel URBasic

Set the comunication with the robot

In [ ]:
import sys
from URBasic import ISCoin

robot_ip = ""
robot_opened_gripper_size_mm = 40. # Max oppenning of gripper

iscoin = ISCoin(robot_ip, robot_opened_gripper_size_mm)

if not iscoin.isInitOk:
    print("Error robot not initialised")
    sys.exit(1)

## Test motion

It is possible to store robot position in two ways;

1. Tool Center Position(TCP) Position (TCP6D);
    - This position is center on the tool position by encoding the cartesian coordinate (**x**,**y**,**z**) and the orientation along the axis (**rx**,**ry**,**rz**) of the tool.
    - **warning**: Will it make easy to compute the tool position, the robot may have multiple way to reach the say tool position.
    - *Application*: When we need to know the tool position without error.

2. Joint Position (Joint6D);
    - A robot arm is composed of different motor called joint. By setting each of these robot to a specific position the robot can reach a position. In this configuration the robot will have only one possibility to reach the position as it encode the rotation of each joint of the robot.
    - **Warning**: Will it make the position of the robot know without any doubes, it is more difficult to know the end point of the robot -> TCP
    - *Application*: For safe position, and calibration position.



In [ ]:
from URBasic import TCP6D
from URBasic import Joint6D

iscoin.robot_control.reset_error()

# initial_pos   = TCP6D.createFromMetersRadians(x, y, z, rx, ry, rz)    # Coordinate of the tools
# initial_joint = Joint6D.createFromRadians(j1, j2, j3, j4, j5, j6)     # Value of the robot joints

""" 
movel(self, pose : TCP6D, a : float = 1.2, v : float = 0.25, t : float = 0, r : float = 0, wait : bool = True)
    Move to position (linear in tool-space)
    Parameters:
    pose: target pose [m]
    a:    tool acceleration [m/s^2]
    v:    tool speed [m/s]
    t:    time [S]
    r:    blend radius [m]
    wait: function return when movement is finished

movej(self, joints : Joint6D, a : float = 1.4, v : float = 1.05, t : float = 0, r : float = 0, wait : bool = True)
    Move to position (linear in joint-space)
    Parameters:
    joints:    joint positions [rad]
    a:    joint acceleration of leading axis [rad/s^2]
    v:    joint speed of leading axis [rad/s]
    t:    time [S]
    r:    blend radius [m]
    wait: function return when movement is finished
"""

motion = [
    # # 1. Move robot to a safe home position (joint space)
    # Joint6D.createFromRadians(
    #     0.0, -1.57, 1.57, 0.0, 1.57, 0.0
    # ),

    # # 2. Move above the pick location (TCP space)
    # TCP6D.createFromMetersRadians(
    #     0.40, 0.10, 0.30,   # x, y, z in meters
    #     3.14, 0.0, 0.0      # rx, ry, rz in radians
    # ),

    # # 3. Move down to the pick point (TCP space)
    # TCP6D.createFromMetersRadians(
    #     0.40, 0.10, 0.10,
    #     3.14, 0.0, 0.0
    # ),

    # # 4. Lift back up (TCP space)
    # TCP6D.createFromMetersRadians(
    #     0.40, 0.10, 0.30,
    #     3.14, 0.0, 0.0
    # ),

    # # 5. Move to a different joint configuration (joint space)
    # Joint6D.createFromRadians(
    #     -0.5, -1.2, 1.8, -0.3, 1.4, 0.2
    # ),

    # # 6. Move to a place location (TCP space)
    # TCP6D.createFromMetersRadians(
    #     0.20, -0.25, 0.15,
    #     3.14, 0.0, 1.57
    # ),

    # # 7. Return to a safe home position (joint space)
    # Joint6D.createFromRadians(
    #     0.0, -1.57, 1.57, 0.0, 1.57, 0.0
    # ),
]

robot_arm = iscoin.robot_control
for m in motion:
    if isinstance(m, TCP6D):
        robot_arm.movel(m)
    elif isinstance(m, Joint6D):
        robot_arm.movej(m)

## Test gripper

**Important**: Before using the gripper it should be activated.

**Important**: When the gripper will not be used you can deactivat it.

In [ ]:
from time import sleep

robot_gripper = iscoin.gripper

robot_gripper.activate()

robot_gripper.waitUntilActive()
if not robot_gripper.isActivated():
  print("Error the gripper was not activated")

In [ ]:
print("Actual gripper opening:\t" + str(robot_opened_gripper_size_mm) + " [mm]")
print("Approximate resolution:\t" + str(robot_opened_gripper_size_mm/255) + " [mm] per unit")

Actual gripper opening:	40.0 [mm]
Approximate resolution:	0.1568627450980392 [mm] per unit


Actual gripper opening: 40 mm  
Approximate resolution: ~0.2 mm per unit

### Basic Command Gripper

In [ ]:
"""
  pos: Position to move to (0-255, 0 being open (50mm), 255 being closed (0mm)). Around 0.2mm per unit.
  speed: Speed of the gripper (0-255, 0 being slowest) default = 255
  force: Force of the gripper (0-255, 0 being weakest) default = 50
"""

if not robot_gripper.move(pos=155, speed=255, force=50): # 20mm -> 255 - (20/0.2) = 155
  print("Failed to move gripper")
sleep(1) # Allow time for the gripper to move

if not robot_gripper.isMoving():
  print("The gripper is still moving")
  robot_gripper.waitUntilStopped() # Alternative to the sleep

if not robot_gripper.open(speed=255, force=50):
  print("Failed to open gripper")
else:
  robot_gripper.waitUntilStopped()
  if not robot_gripper.close(speed=255, force=50):
    print("Failed to close gripper")
robot_gripper.waitUntilStopped()

### Gripper Object Detection

In [ ]:
robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

if not robot_gripper.hasDetectedObject():
    print("The gripper did not catch an object")
else:
    print("The gripper did catch an object")

if robot_gripper.isMoving():
    print("I cannot measure the object dimension as the gripper move.")
else:
    print("The gripper catch an object of dimension " + str(robot_gripper.getEstimatedPositionMm())) + "mm")

sleep(10)

robot_gripper.open(speed=255, force=50)
robot_gripper.waitUntilStopped()

# This will close the gripper and say if the object match the expectations
if not robot_gripper.closeAndCheckSize(expected_size_mm=15., plus_margin_mm=2.5, minus_margin_mm=3.2, speed=255, force=50):
    print("The gripper did not catch the expected object:     " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
else:
    print("The gripper did catch a potential expected object: " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
# robot_gripper.waitUnitilStopped() is incorporated into closeAndCheckSize function

sleep(10)

robot_gripper.open(speed=255, force=50)
robot_gripper.waitUntilStopped()
robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

# This will open the gripper and say if the object match the expectations
if not robot_gripper.openAndCheckSize(expected_size_mm=15., plus_margin_mm=2.5, minus_margin_mm=3.2, speed=255, force=50)
    print("The gripper did not catch the expected object:     " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
else:
    print("The gripper did catch a potential expected object: " + str(robot_gripper.getEstimatedPositionMm())) + " mm")
# robot_gripper.waitUnitilStopped() is incorporated into openAndCheckSize function

sleep(10)

robot_gripper.close(speed=255, force=50)
robot_gripper.waitUntilStopped()

In [ ]:
robot_gripper.move(pos=55, speed=255, force=50) # 40mm -> 255 - (40/0.2) = 55
robot_gripper.waitUntilStopped()
robot_gripper.deactivate()